PACKAGE INSTALLATIONS AND IMPORTS

In [3]:
import pandas as pd

In [4]:
#Packages required for cleaning the data
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from sklearn.model_selection import train_test_split

DATA SET

In [9]:
# Path to the data set.Device specific.
dataset_path = '/Users/unnimaya/Documents/Projectexperiments/AIRLINEDATASET/usairlinetweets.csv'
# Load the dataset
df_airline = pd.read_csv(dataset_path)
print(df_airline.head())

             tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   
3  570301031407624196          negative                        1.0000   
4  570300817074462722          negative                        1.0000   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                     0.0000  Virgin America   
2            NaN                        NaN  Virgin America   
3     Bad Flight                     0.7033  Virgin America   
4     Can't Tell                     1.0000  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0                    NaN     cairdin                 NaN              0   
1                    NaN    jnar

In [10]:
df = df_airline[["airline_sentiment", "text"]]
df

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [11]:
df["airline_sentiment"].value_counts()
#checking the class distribution.

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [12]:

# Split the data into train and test sets
class_label = 'airline_sentiment'
# Taking 200 data points from each class for the test set
test_df_class_1 = df[df[class_label] == 'negative'].sample(n=200, random_state=42)
test_df_class_2 = df[df[class_label] == 'neutral'].sample(n=200, random_state=42)
test_df_class_3 = df[df[class_label] == 'positive'].sample(n=200, random_state=42)
# Concatenate the sampled data frames to create the test set
test_df_unmodified = pd.concat([test_df_class_1, test_df_class_2, test_df_class_3])
# Create the train set by removing the test set samples
train_df_unmodified = df.drop(test_df_unmodified.index)
print("Test set class distribution:\n", test_df_unmodified[class_label].value_counts())
print("Train set class distribution:\n", train_df_unmodified[class_label].value_counts())


Test set class distribution:
 airline_sentiment
negative    200
neutral     200
positive    200
Name: count, dtype: int64
Train set class distribution:
 airline_sentiment
negative    8978
neutral     2899
positive    2163
Name: count, dtype: int64


In [13]:
test_df_unmodified.index

Index([ 1151,  9111,  3326, 10891, 11322,  3709,  5261, 10060, 12446,  3089,
       ...
       11705, 13422,  2716,  3071, 13504,  4489,  8271, 11795,  7294,  8203],
      dtype='int64', length=600)

In [14]:
test_df_class_1.index

Index([ 1151,  9111,  3326, 10891, 11322,  3709,  5261, 10060, 12446,  3089,
       ...
       13324, 11935,  9346, 11781,  3022, 10759, 11149,  7523, 12754,   667],
      dtype='int64', length=200)

In [15]:
train_df_unmodified.shape

(14040, 2)

In [16]:
test_df_unmodified.shape

(600, 2)

In [17]:
df.shape

(14640, 2)

In [18]:
#Cleaning the data
lemmatizer = WordNetLemmatizer()#initialize lemmatizer
stop_words = set(stopwords.words('english'))#initialize stopwords

def text_cleaning(text):
    text = text.lower()#converting the text to lowercase
    text = re.sub(r'[^\w\s]', '', text)#punctuation removal using regular expression
    text = re.sub(r'\d+', '', text)#number removal
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]#stop word removal and lemmatization
    cleaned_text = ' '.join(tokens)#joins the tokens to form cleaned sentence   
    return cleaned_text

In [19]:
%%time
#cleaning the train data
train_df_unmodified['text'] = train_df_unmodified['text'].apply(text_cleaning)

CPU times: user 2min 30s, sys: 5.06 s, total: 2min 35s
Wall time: 2min 52s


In [20]:
train_df_unmodified

,airline_sentiment,text
1,positive,virginamerica plus youve added commercial expe...
2,neutral,virginamerica didnt today must mean need take ...
3,negative,virginamerica really aggressive blast obnoxiou...
4,negative,virginamerica really big bad thing
5,negative,virginamerica seriously would pay flight seat ...
...,...,...
14635,positive,americanair thank got different flight chicago
14636,negative,americanair leaving minute late flight warning...
14637,neutral,americanair please bring american airline blac...
14638,negative,americanair money change flight dont answer ph...


In [21]:
%%time
#cleaning the test data
test_df_unmodified['text'] = test_df_unmodified['text'].apply(text_cleaning)

CPU times: user 6.36 s, sys: 234 ms, total: 6.59 s
Wall time: 7.74 s


In [22]:
test_df_unmodified

,airline_sentiment,text
1151,negative,united gate c iad u released passenger board w...
9111,negative,usairways connecting charlotte houston mechani...
3326,negative,united installed working kicked hour wouldnt l...
10891,negative,usairways flight fll told take train back pbi ...
11322,negative,usairways loosing lot business using barclays ...
...,...,...
4489,positive,southwestair left ipad seat pocket filed lost ...
8271,positive,jetblue thatd nice hoping rack enough mile tak...
11795,positive,usairways customer service finest
7294,positive,jetblue awesome


OVERSAMPLING

In [23]:
from sklearn.utils import resample#importing resample function from sklearn
class_counts = train_df_unmodified['airline_sentiment'].value_counts()#Getting the number of data points in each class.
max_class_count = class_counts.max() # Getting the number of samples in the majority class
#seperating each class to dataframes
negative_df = train_df_unmodified[train_df_unmodified['airline_sentiment'] == 'negative']
neutral_df = train_df_unmodified[train_df_unmodified['airline_sentiment'] == 'neutral']
positive_df = train_df_unmodified[train_df_unmodified['airline_sentiment'] == 'positive']

# Resampling the neutral and positive DataFrames to match the number of samples in the negative class
neutral_df_oversampled = resample(neutral_df,
                                  replace=True,     # sampling with replacement
                                  n_samples=max_class_count,    # giving max count as majority class count
                                  random_state=42)  # reproducible results

positive_df_oversampled = resample(positive_df,
                                   replace=True,     # sample with replacement
                                   n_samples=max_class_count,   # giving max count as majority class count
                                   random_state=42)  # reproducible results

#combining them to make an undersampled data frame
oversampled_df = pd.concat([negative_df, neutral_df_oversampled, positive_df_oversampled])

#shuffling the undersampled dataframe
oversampled_df = oversampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Oversampled DataFrame:")
print(oversampled_df['airline_sentiment'].value_counts())
#From here on training df= oversampled df, testing df= test_df_unmodified

Oversampled DataFrame:
airline_sentiment
negative    8978
neutral     8978
positive    8978
Name: count, dtype: int64


VADER

In [ ]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


In [24]:

analyzer = SentimentIntensityAnalyzer() # Sentiment analyzer initialization
#Function to obtain the sentiment scores using VADER
def obtain_sentiment_scores(texts):
    scores = []
    for text in texts:
        score = analyzer.polarity_scores(text)#calculating polarity scores
        scores.append(score['compound'])  # Using the compound score
    return scores

# Convert sentiment scores to binary predictions
def obtain_binary_predictions(scores):
    predictions = []
    for score in scores:
        if score >= 0.05:
            predictions.append('positive')
        elif score <= -0.05:
            predictions.append('negative')
        else:
            predictions.append('neutral')
    return predictions


In [25]:
%%time

sentiment_scores_test = obtain_sentiment_scores(oversampled_df['text'])#calling the obtain_sentiment_scores function
y_pred_test = obtain_binary_predictions(sentiment_scores_test).# getting the predictions

CPU times: user 947 ms, sys: 25.9 ms, total: 972 ms
Wall time: 1.41 s


In [26]:
%%time
# Convert labels to binary for each sentiment class
#For test set
y_test_binary_positive = (oversampled_df['airline_sentiment'] == 'positive').astype(int)
y_test_binary_negative = (oversampled_df['airline_sentiment'] == 'negative').astype(int)
y_test_binary_neutral = (oversampled_df['airline_sentiment'] == 'neutral').astype(int)
#For the predictions
y_pred_binary_positive = (pd.Series(y_pred_test) == 'positive').astype(int)
y_pred_binary_negative = (pd.Series(y_pred_test) == 'negative').astype(int)
y_pred_binary_neutral = (pd.Series(y_pred_test) == 'neutral').astype(int)

CPU times: user 10 ms, sys: 4.23 ms, total: 14.3 ms
Wall time: 13.5 ms


In [31]:
%%time
#f1 score calculation
f1_score_positive = f1_score(y_test_binary_positive, y_pred_binary_positive)#For positive
print("Positive F1 Score:", f1_score_positive)
f1_score_negative = f1_score(y_test_binary_negative, y_pred_binary_negative)#For negative
print("Negative F1 Score:", f1_score_negative)
f1_score_neutral = f1_score(y_test_binary_neutral, y_pred_binary_neutral)#For neutral
print("Neutral F1 Score:", f1_score_neutral)

Positive F1 Score: 0.6133302547525591
Negative F1 Score: 0.4927557768036258
Neutral F1 Score: 0.42903695486375926
Average f1 score: 0.5117076621399814
CPU times: user 24.6 ms, sys: 15 ms, total: 39.5 ms
Wall time: 96.2 ms


In [29]:
%%time
# Calculating AUC score
auc_score_positive = roc_auc_score(y_test_binary_positive, sentiment_scores_test)#For positive
print("Positive AUC Score:", auc_score_positive)
auc_score_negative = roc_auc_score(y_test_binary_negative, sentiment_scores_test)#For negative
print("Negative AUC Score:", auc_score_negative)
auc_score_neutral = roc_auc_score(y_test_binary_neutral, sentiment_scores_test)#For neutral
print("Neutral AUC Score:", auc_score_neutral)
average_auc_score = (auc_score_positive + auc_score_negative + auc_score_neutral) / 3
print("Average auc score:",average_auc_score)

Positive AUC Score: 0.7944405642495027
Negative AUC Score: 0.27219088394635715
Neutral AUC Score: 0.43336855180414036
Average auc score: 0.5000000000000001
CPU times: user 22 ms, sys: 2.12 ms, total: 24.1 ms
Wall time: 26.2 ms


In [101]:
%%time
# Calculating accuracy score for the test set
accuracy = accuracy_score(oversampled_df['airline_sentiment'], y_pred_test)#overall accuracy
print("Overall Accuracy:", accuracy)
accuracy_positive = accuracy_score( y_test_binary_positive, y_pred_binary_positive )# accuracy for positive class
print("Positive Accuracy:", accuracy_positive)
accuracy_negative = accuracy_score( y_test_binary_negative, y_pred_binary_negative )# accuracy for negative class
print("Negative Accuracy:", accuracy_negative)
accuracy_neutral = accuracy_score( y_test_binary_neutral, y_pred_binary_neutral )#accuracy for neutral class
print("Neutral Accuracy:", accuracy_neutral)

Overall Accuracy: 0.5338605480062375
Positive Accuracy: 0.6269399272295241
Negative Accuracy: 0.7465285512734833
Neutral Accuracy: 0.6942526175094675
CPU times: user 32.1 ms, sys: 2.76 ms, total: 34.9 ms
Wall time: 35.8 ms


AFINN

In [34]:
from sklearn.metrics import accuracy_score
from afinn import Afinn
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize

In [35]:
%%time
afinn = Afinn()
# Function to compute AFINN scores
def get_afinn_scores(text):
    score = afinn.score(text)
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

CPU times: user 48.2 ms, sys: 13.5 ms, total: 61.8 ms
Wall time: 112 ms


In [36]:
#Getting the scores my applying the get_afinn_scores function
oversampled_df['predicted_sentiment'] = oversampled_df['text'].apply(get_afinn_scores)

In [37]:
%%time
#Prediction 
df_test = oversampled_df['text'].to_frame()
df_test['predicted_sentiment'] = df_test['text'].apply(get_afinn_scores)
# Accuracy calculation
accuracy = accuracy_score(oversampled_df['airline_sentiment'], df_test['predicted_sentiment'])
print(f"Accuracy: {accuracy}")

Accuracy: 0.5556545630058662
CPU times: user 4.33 s, sys: 17 ms, total: 4.35 s
Wall time: 4.37 s


In [38]:
%%time
#Binarising for auc calculation
y_test_bin = label_binarize(oversampled_df['airline_sentiment'], classes=['negative', 'neutral', 'positive'])
y_pred_bin = label_binarize(df_test['predicted_sentiment'], classes=['negative', 'neutral', 'positive'])
#auc calculation
try:
    auc = roc_auc_score(y_test_bin, y_pred_bin, average='macro', multi_class='ovo')
    print(f"AUC: {auc}")
except ValueError:
    print("AUC cannot be computed for single class predictions.")

AUC: 0.6667409222543995
CPU times: user 39.3 ms, sys: 5.41 ms, total: 44.7 ms
Wall time: 62.8 ms


In [40]:
%%time
from sklearn.metrics import f1_score
#Calculating f1score for afinn in undersampled data
f1_classwise = f1_score(oversampled_df['airline_sentiment'], df_test['predicted_sentiment'], average=None, labels=['positive', 'negative', 'neutral'])

#Calculating average f1score
f1_average = f1_score(oversampled_df['airline_sentiment'], df_test['predicted_sentiment'], average='macro', labels=['positive', 'negative', 'neutral'])

print("F1 Scores:")
print(f"F1 Score for Positive: {f1_classwise[0]}")#class positive
print(f"F1 Score for Negative: {f1_classwise[1]}")#class negative
print(f"F1 Score for Neutral: {f1_classwise[2]}")#class neutral
print(f"Average F1 Score: {f1_average}")

F1 Scores:
F1 Score for Positive: 0.6336017265709305
F1 Score for Negative: 0.5286463949392567
F1 Score for Neutral: 0.4609468574670617
Average F1 Score: 0.5410649929924163
CPU times: user 262 ms, sys: 55.7 ms, total: 318 ms
Wall time: 1.38 s


TextBlob

In [43]:

from textblob import TextBlob
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize


In [44]:
# Function to compute TextBlob sentiment scores
def get_textblob_scores(text):
    analysis = TextBlob(text) #calculating polarity scores for textblob
    score = analysis.sentiment.polarity
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

In [45]:
%%time
# Apply TextBlob to get sentiment labels
oversampled_df['predicted_sentiment'] = oversampled_df['text'].apply(get_textblob_scores)

CPU times: user 1.9 s, sys: 132 ms, total: 2.03 s
Wall time: 2.78 s


In [46]:
%%time
#prediction calculation
df_test = oversampled_df['text'].to_frame()
df_test['predicted_sentiment'] = df_test['text'].apply(get_textblob_scores)#applying the function to get textblob scores
# Accuracy calculation for textBlob
accuracy_textblob = accuracy_score(oversampled_df['airline_sentiment'], df_test['predicted_sentiment'])
print(f"Accuracy: {accuracy_textblob}")

Accuracy: 0.5531670008168115
CPU times: user 1.83 s, sys: 25.3 ms, total: 1.86 s
Wall time: 1.89 s


In [47]:
%%time
#auc for textblob on oversampled data
y_test_bin_t = label_binarize(oversampled_df['airline_sentiment'], classes=['negative', 'neutral', 'positive'])[:, 2]  # 1 for 'positive', 0 for others
y_pred_bin_t = label_binarize(df_test['predicted_sentiment'], classes=['negative', 'neutral', 'positive'])[:, 2]
#multiclass auc calculation 
try:
    auc = roc_auc_score(y_test_bin_t, y_pred_bin_t)#we throw an exsception if the number of classes is 1 as this is a multiclass auc calculation
    print(f"AUC: {auc}")
except ValueError:
    print("AUC cannot be computed for single class predictions.")

AUC: 0.7198151035865448
CPU times: user 31.4 ms, sys: 15.9 ms, total: 47.3 ms
Wall time: 118 ms


In [48]:
%%time
# Evaluation Metric 3: F1 Score (for each class)
f1_text_blob = f1_score(oversampled_df['airline_sentiment'], df_test['predicted_sentiment'], average=None, labels=['positive', 'negative', 'neutral'])
print(f"F1 Score for Positive: {f1[0]}")#calculating f1score for positive class
print(f"F1 Score for Negative: {f1[1]}")#calculating f1score for negativeclass
print(f"F1 Score for Neutral: {f1[2]}")#calculating f1score for neutral class

F1 Score for Positive: 0.6336017265709305
F1 Score for Negative: 0.5286463949392567
F1 Score for Neutral: 0.4609468574670617
CPU times: user 107 ms, sys: 16.3 ms, total: 123 ms
Wall time: 150 ms


In [49]:
%%time
from sklearn.metrics import f1_score
#getting f1score for textblob in oversampled data
f1_classwise = f1_score(oversampled_df['airline_sentiment'], df_test['predicted_sentiment'], average=None, labels=['positive', 'negative', 'neutral'])
f1_average_textblob = f1_score(oversampled_df['airline_sentiment'], df_test['predicted_sentiment'], average='macro', labels=['positive', 'negative', 'neutral'])

print("F1 Scores:")
print(f"F1 Score for Positive: {f1_classwise[0]}")
print(f"F1 Score for Negative: {f1_classwise[1]}")
print(f"F1 Score for Neutral: {f1_classwise[2]}")
print(f"Average F1 Score: {f1_average_textblob}")

F1 Scores:
F1 Score for Positive: 0.6322703474535936
F1 Score for Negative: 0.44272844272844275
F1 Score for Neutral: 0.5428266069520077
Average F1 Score: 0.5392751323780147
CPU times: user 198 ms, sys: 12.4 ms, total: 211 ms
Wall time: 230 ms


SELF SUPERVISED LEARNING


In [49]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [50]:
%%time
#preprocessing the train data
nltk.download('vader_lexicon')
#initialize sentiment analyser
sid = SentimentIntensityAnalyzer()
train_df, val_df = train_test_split(oversampled_df, test_size=0.2, random_state=42)
test_df= test_df_unmodified
# Obtaining the sentiment labels
def get_sentiment_label(text):
    scores = sid.polarity_scores(text)#Calculating polarity scores
    compound = scores['compound']#calculating compound scores
    if compound >= 0.5:
        return 'positive', compound#assigning the labels according to the calculated compound scores
    elif compound <= -0.5:
        return 'negative', compound
    else:
        return 'neutral', compound

#Psedolabel using different confidence threshold for each class
def pseudo_labeling_vader(df, pos_threshold=0.8, neg_threshold=0.5, neutral_threshold=(0.1, -0.1)):
    pseudo_texts = []
    pseudo_labels = []#list for pseudolabels
    for text in df['text']:  #iterate through the text and get sentiment label and confidence score for each
        sentiment, confidence = get_sentiment_label(text)
        if sentiment == 'positive' and confidence >= pos_threshold:
            pseudo_texts.append(text)#append the text to pseudo texts
            pseudo_labels.append(sentiment)# append the label for the text to pseudolabels
        elif sentiment == 'negative' and confidence <= -neg_threshold:
            pseudo_texts.append(text)
            pseudo_labels.append(sentiment)
        elif sentiment == 'neutral' and neutral_threshold[0] >= confidence >= neutral_threshold[1]:#check for neutral class with its confidence thresholds
            pseudo_texts.append(text)
            pseudo_labels.append(sentiment)
    return pseudo_texts, pseudo_labels

pseudo_texts, pseudo_labels = pseudo_labeling_vader(train_df)#getting pseudo labels

pseudo_df = pd.DataFrame({'text': pseudo_texts, 'airline_sentiment': pseudo_labels})#creating a pseudo labelled dataframe

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(pseudo_df['text'])#vectorising the train set
X_val = vectorizer.transform(val_df['text'])
# Encoding the labels in train and validation set
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(pseudo_df['airline_sentiment'])
y_val = label_encoder.transform(val_df['airline_sentiment'])


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/unnimaya/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


CPU times: user 1.48 s, sys: 23.5 ms, total: 1.5 s
Wall time: 1.53 s


In [51]:
test_df

,airline_sentiment,text,predicted_sentiment
1151,negative,united gate c iad u released passenger board w...,neutral
9111,negative,usairways connecting charlotte houston mechani...,neutral
3326,negative,united installed working kicked hour wouldnt l...,negative
10891,negative,usairways flight fll told take train back pbi ...,neutral
11322,negative,usairways loosing lot business using barclays ...,positive
...,...,...,...
4489,positive,southwestair left ipad seat pocket filed lost ...,negative
8271,positive,jetblue thatd nice hoping rack enough mile tak...,positive
11795,positive,usairways customer service finest,neutral
7294,positive,jetblue awesome,positive


In [52]:
%%time
#test data preprocessing
X_test = vectorizer.transform(test_df['text'])# Vectorization of data
y_test = label_encoder.transform(test_df['airline_sentiment'])#Encoding the labels

CPU times: user 7.37 ms, sys: 6.87 ms, total: 14.2 ms
Wall time: 12.7 ms


In [53]:
%%time 
#training
# Model fitting
svm_model = SVC(probability=True, kernel='linear')#support vector machine with linear kernel
svm_model.fit(X_train, y_train)

CPU times: user 9.28 s, sys: 87.9 ms, total: 9.37 s
Wall time: 9.38 s


SVC(kernel='linear', probability=True)

In [54]:
%%time
# Validation
val_predictions = svm_model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)#calculating the validation accuracy for SSL
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.5318359012437349
CPU times: user 973 ms, sys: 9.14 ms, total: 982 ms
Wall time: 987 ms


In [55]:
%%time
# Testing
test_predictions = svm_model.predict(X_test)#predicting using SSL
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy}")#Calculating test accuracy for SSL

Test Accuracy: 0.5583333333333333
CPU times: user 142 ms, sys: 3.66 ms, total: 145 ms
Wall time: 145 ms


BAGGING WITH KNN

In [57]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
%%time
#defining test and train sets
X_train= oversampled_df['text']
y_train=oversampled_df['airline_sentiment']
X_test=test_df_unmodified['text']
y_test=test_df_unmodified['airline_sentiment']

CPU times: user 343 µs, sys: 2.01 ms, total: 2.35 ms
Wall time: 2.38 ms


In [59]:
%%time

def tokenize(text):
    return word_tokenize(text)#tokenizing

# Pipeline definition for vectorization and classifier
pipeline_bagging = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('bagging_knn', BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=3), n_estimators=10, random_state=42))
])

CPU times: user 133 µs, sys: 667 µs, total: 800 µs
Wall time: 806 µs


In [60]:
%%time
#training
pipeline_bagging.fit(X_train, y_train)#fitting the pipeline

/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 4min 19s, sys: 5.02 s, total: 4min 24s
Wall time: 4min 29s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x15b069f30>)),
                ('bagging_knn',
                 BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=3),
                                   random_state=42))])

In [61]:
%%time
#testing
y_pred_bagging = pipeline_bagging.predict(X_test)#predict bagging knn
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)#calculating accuracy of bagging knn
print("Bagging KNN Accuracy:", accuracy_bagging)

Bagging KNN Accuracy: 0.6066666666666667
CPU times: user 8.95 s, sys: 319 ms, total: 9.27 s
Wall time: 9.36 s


BOOSTING WITH DECISION TREES

In [62]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
%%time
X_train= oversampled_df['text']
y_train=oversampled_df['airline_sentiment']
X_test=test_df_unmodified['text']
y_test=test_df_unmodified['airline_sentiment']

CPU times: user 46 µs, sys: 389 µs, total: 435 µs
Wall time: 1.27 ms


In [64]:
def tokenize(text):
    return word_tokenize(text)

# Boosting with Decision Tree -pipeline deifnition
pipeline_boosting = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('boost_tree', AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3), n_estimators=50, random_state=42))
])

In [65]:
%%time
#training
pipeline_boosting.fit(X_train, y_train)#model fitting

/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


CPU times: user 4min 20s, sys: 3.72 s, total: 4min 24s
Wall time: 4min 27s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x1684039a0>)),
                ('boost_tree',
                 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3),
                                    random_state=42))])

In [66]:
%%time
#testing
y_pred_boosting = pipeline_boosting.predict(X_test)#running predictions
accuracy_boosting = accuracy_score(y_test, y_pred_boosting)#accuracy score calculation
print("Boosting with Decision Tree Accuracy:", accuracy_boosting)

Boosting with Decision Tree Accuracy: 0.7066666666666667
CPU times: user 5.8 s, sys: 63.5 ms, total: 5.87 s
Wall time: 5.91 s


In [ ]:
VOTING WITH LOGISTIC REGRESSION, SVM AND DECISION TREE

In [67]:
#ensemble model- voting using logistic regression, svc and decision tree
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
X_train= oversampled_df['text']
y_train=oversampled_df['airline_sentiment']
X_test=test_df_unmodified['text']
y_test=test_df_unmodified['airline_sentiment']

In [69]:
%%time
#Function for tokenizing the data.
def tokenize(text):
    return word_tokenize(text)

# Defining the individual models for voting
model1 = LogisticRegression(random_state=42, max_iter=1000)
model2 = SVC(probability=True, random_state=42)
model3 = DecisionTreeClassifier(random_state=42)

# Voting Classifier: voting category is soft 
voting_classifier = VotingClassifier(estimators=[
    ('lr', model1),#lr for linear regression
    ('svc', model2),#svc for support vector machine
    ('dt', model3)#dt for decision tree
], voting='soft')

# Pipeline creation.
pipeline_voting = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('voting', voting_classifier)
])


CPU times: user 183 µs, sys: 327 µs, total: 510 µs
Wall time: 1.17 ms


In [70]:
%%time
# Train the pipeline
pipeline_voting.fit(X_train, y_train)#pipeline fitting

/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 9min 40s, sys: 8 s, total: 9min 48s
Wall time: 9min 47s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x15d8db2e0>)),
                ('voting',
                 VotingClassifier(estimators=[('lr',
                                               LogisticRegression(max_iter=1000,
                                                                  random_state=42)),
                                              ('svc',
                                               SVC(probability=True,
                                                   random_state=42)),
                                              ('dt',
                                               DecisionTreeClassifier(random_state=42))],
                                  voting='soft'))])

In [71]:
%%time
# Running predictions.
y_pred_voting = pipeline_voting.predict(X_test)
accuracy_voting = accuracy_score(y_test, y_pred_voting)#Accuracy calculation for voting algorithm
print("Voting Classifier Accuracy:", accuracy_voting)

Voting Classifier Accuracy: 0.73
CPU times: user 6.36 s, sys: 345 ms, total: 6.7 s
Wall time: 6.74 s
